# *args*

In [1]:
from argparse import Namespace

In [2]:
args = Namespace(
    # 날짜와 경로 정보
    frequency_cutoff = 25,
    model_state_file='model.pth',
    review_csv='/mnt/nas_storage3/shpark/practice/data/yelp/reviews_with_splits_lite.csv',
    save_dir='/mnt/nas_storage3/shpark/practice/training/torch/model/yelp/',
    # 모델 하이퍼파라미터 없음
    # 훈련 하이퍼파라미터
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs = 100,
    seed = 1337,
    # 실행 옵션 생략
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
)

# *dataset*

In [3]:
import collections
import numpy as np
import pandas as pd
import re

In [4]:
# 별점 기준으로 나누어 훈련, 검증, 테스트를 만듭니다.
by_rating = collections.defaultdict(list)
review_subset = pd.read_csv(args.review_csv, header=None, names=['rating', 'review'])

In [5]:
review_subset.head()

,rating,review
rating,review,split
negative,all i can say is that a i had no other option ...,train
negative,i went here once when my long time stylist mov...,train
negative,i don t know why i stopped here for lunch this...,train
negative,did i order the wrong thing ? or maybe it was ...,train


In [6]:
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())

In [7]:
by_rating

defaultdict(list,
            {'review': [{'rating': 'review', 'review': 'split'}],
             'all i can say is that a i had no other option then to go with time warner cable for my internet service n nand b . . . . n nif you don t have a local number it will be an absolute nightmare to pay your bill ! you can go through the ring of automated messages only to get to a live person after min then be told they will charge you so you must go through the automated service . . . . repeat . i cringe when it comes time to pay my bill each month . n nif you are lucky enough to avoid using them i suggest you do so ! ': [{'rating': 'all i can say is that a i had no other option then to go with time warner cable for my internet service n nand b . . . . n nif you don t have a local number it will be an absolute nightmare to pay your bill ! you can go through the ring of automated messages only to get to a live person after min then be told they will charge you so you must go through the automate

In [8]:
# 분할 데이터를 만듭니다.
final_list = []
np.random.seed(args.seed)

In [9]:
for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)
    n_total=len(item_list)
    n_train=int(args.train_proportion * n_total)
    n_val=int(args.val_proportion * n_total)
    n_test=int(args.test_proportion * n_total)
    
    # 데이터 포인터에 분할 속성을 추가합니다
    for item in item_list[:n_train]:
        item['split']='train'
    
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'
    
    # 최종 리스트에 추가합니다
    final_list.extend(item_list)

final_reviews = pd.DataFrame(final_list)

In [10]:
final_reviews.head(10)

,rating,review,split
0,,train,NaN
1,! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! disgusting ! ...,train,NaN
2,.,train,NaN
3,. . .,train,NaN
4,. . . .,train,NaN
5,. . . . . . . where are the chopsticks ? n na...,train,NaN
6,. . . . it s the land of misfit toys . n nthe...,val,NaN
7,". . . . my hair looks fine , but i swear to g...",val,NaN
8,. . . and now i know what italian food is sup...,train,NaN
9,. . . and they re closed . couldn t have happ...,train,NaN


In [11]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[a-zA-Z.,!?]+", r" ", text)

    return text

final_reviews.review = final_reviews.review.apply(preprocess_text)

# *pytorch dataset*

In [12]:
from torch.utils.data import Dataset

In [17]:
class ReviewDataset(Dataset):
    def _init_(self, review_df, vectorizer):
        """
        매개변수:
            review_df (pandas.DataFrame): 데이터셋
            vectorizer (ReviewVectorizer): ReviewVectorizer 객체
        """
        self.review_df = review_df
        self._vectorizer = vectorizer
        
        self.train_df = self.review_df[self.review_df.split=='train']
        self.train_size = len(self.train_df)
        
        self.val_df - self.review_df[self.review_df.split=='val']
        self.validation_size = len(self.val_df)
        
        self.test_df - self.review_df[self.review_df.split=='test']
        self.test_size = len(self.test_df)
        
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}
        
        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """
        데이터셋을 로드하고 새로운 Reviewvectorizer 객체를 만듭니다.
        
        parameters:
            review_csv (str): GoE499121 
        returns:
            ReviewDataset의 인스턴스
        """
        review_df = pd.read_csv(review_csv)
        return cls(review_df, ReviewVectorizer.from_dataframe(review_df))
    
    def get_vectorizer(self):
        """
        ReviewVectorizer 객체를 반환합니다.
        """
        return self._vectorizer
    
    def set_split(self, split="train"):
        """
        데이터프레임에 있는 열을 사용해 분할 새트를 선택합니다.
        
        parameters:
            split (str): "train", "val", "test" 중 하나
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    
    def _len_(self):
        return self._target_size

    def _getitem_(self, index):
        """
        파이토치 데이터셋의 주요 진입 메서드
        
        parameters:
            index (int): 데이터 포인트의 인덱스
        returns:
            데이터 포인트의 특성(x_data)과 레이블(y_target)로 이루어진 딕셔너리
        """
        row = self._target_df.iloc[index]
        
        review_vector = self._vertorizer.vectorize(row.review)
        rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)
        
        return {'x_data': review_vector,
                'y_target': rating_index}
    
    def get_num_batches(self, batch_size):
        """
        배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다.

        parameters:
            batch_size(int)
        returns:
            배치 개수
        """
        return len(self) // batch_size

# *Vocabulary*

In [19]:
class Vocabulary(object):
    """
    매핑을 위해 백스트를 처리하고 어휘 사전을 만드는 클래스
    """
    
    def __init_(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        parameters:
            token_to_idx (dict): 기존 토큰 - 인덱스 매핑 딕셔너리
            add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
            unk_token (str): Vocabulary에 추가할 UNK 토큰
        """

        if token_to_idx is None:
            token_to_idx = {}
    
        self._token_to_idx - token_to_idx
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token

        self.unk_index = -1
        
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    
    def to_serializable(self):
        """
        직렬화할 수 있는 딕셔너리를 반환합니다.
        """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        """
        직렬화된 딕셔너리에서 Vocabulary 객체를 만듭니다.
        """
        return cls(**contents)

    def add_token(self, token):
        """토큰을 기반으로 매핑 딕셔너리를 업데이트합니다
        
        parameters:
            token (str): Vocabulary에 추가할 토큰
        returns:
            index (int): 토큰에 상응하는 정수
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        """
        토큰에 대응하는 인덱스를 추출합니다.
        토큰이 없으면 UNK 인덱스를 반환합니다.
        
        parameters:
            token (str): 찾을 토큰
        returns:
            index (int): 토큰에 해당하는 인덱스
        note:
            UNK 토큰을 사용하려면 (Vocabulary에 추가하기 위해)
            unk_index 가 0보다 커야 합니다.
        """
        if self.add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    
    def lookup_index(self, index):
        """
        인덱스에 해당하는 토큰을 반환합니다.

        parameters:
            index (int): 찾을 인덱스

        returns:
            token (str): 인덱스에 해당하는 토큰

        error:
            KeyError: 인덱스가 Vocabulary에 없을 때 발생합니다.
        """

        if index not in self._idx_to_token:
            raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
        return self.idx_to_token[index]
    
    def _str_(self):
        return "<Vocabulary(size = %d)>" % len(self)

    def _len_(self):
        return len(self._token_to_idx)

# *Vectorizer*